In [74]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [75]:
rawdata= pd.read_csv('resources/clean_data.csv')

In [76]:
rawdata.head()

,Unnamed: 0,City,Business Name,Address,Phone Number,Website,Google Map ID,Review Text,Timestamp,Stars,TrueTimestamp
0,0,adrej,BMCE BANK El Wafaa,"225C+WRW, Fès 30050",NaN,https://business.google.com/create?fp=34034433...,NaN,GAb est sale\r\nPrès d'entrée d'immeuble et bo...,il y a 2 ans,1,2023-03-14 21:22:17.359491
1,1,adrej,BMCE BANK El Wafaa,"225C+WRW, Fès 30050",NaN,https://business.google.com/create?fp=34034433...,NaN,Chef d'agence (une femme) est une hypocrite.,il y a 9 ans,2,2016-03-15 21:22:17.359491
2,2,adrej,BMCE BANK El Wafaa,"225C+WRW, Fès 30050",NaN,https://business.google.com/create?fp=34034433...,NaN,Bonne services,il y a 4 ans,4,2021-03-14 21:22:17.359491
3,3,adrej,BMCE BANK El Wafaa,"225C+WRW, Fès 30050",NaN,https://business.google.com/create?fp=34034433...,NaN,le pire service que j'ai jamais vu dans une ba...,il y a 2 ans,1,2023-03-14 21:22:17.359491
4,4,adrej,BMCE BANK El Wafaa,"225C+WRW, Fès 30050",NaN,https://business.google.com/create?fp=34034433...,NaN,No review text found,il y a 8 mois,5,2024-07-16 21:22:17.359491


In [78]:
#remove column
rawdata= rawdata.drop(columns=['Address','Timestamp',"Phone Number","Unnamed: 0",'Website','Google Map ID'])

remove rows where there is nor review text

In [79]:
rawdata = rawdata[rawdata["Review Text"] != "No review text"]
rawdata.head()

,City,Business Name,Review Text,Stars,TrueTimestamp
0,adrej,BMCE BANK El Wafaa,GAb est sale\r\nPrès d'entrée d'immeuble et bo...,1,2023-03-14 21:22:17.359491
1,adrej,BMCE BANK El Wafaa,Chef d'agence (une femme) est une hypocrite.,2,2016-03-15 21:22:17.359491
2,adrej,BMCE BANK El Wafaa,Bonne services,4,2021-03-14 21:22:17.359491
3,adrej,BMCE BANK El Wafaa,le pire service que j'ai jamais vu dans une ba...,1,2023-03-14 21:22:17.359491
4,adrej,BMCE BANK El Wafaa,No review text found,5,2024-07-16 21:22:17.359491


In [80]:
#Making sure every rating is within the range 1-5
rawdata["Stars"].describe()

count    13981.000000
mean         2.686217
std          1.772554
min          1.000000
25%          1.000000
50%          2.000000
75%          5.000000
max          5.000000
Name: Stars, dtype: float64

Let's work with only few big cities

In [81]:
# List of valid Moroccan cities
valid_cities = [
    "agadir","ifrane","dakhla","fes","errachidia","casablanca","berkane","azilal","harhoura","el+jadida",
]

rawdata["City"] = rawdata["City"].str.lower().str.strip()

# Remove rows where the city is not in the valid list
rawdata = rawdata[rawdata["City"].isin(valid_cities)]
list(rawdata["City"].unique())


['agadir',
 'azilal',
 'berkane',
 'casablanca',
 'dakhla',
 'el+jadida',
 'errachidia',
 'fes',
 'harhoura',
 'ifrane']

In [82]:
rawdata.describe()

,Stars
count,5235.000000
mean,2.584145
std,1.761890
min,1.000000
25%,1.000000
50%,1.000000
75%,5.000000
max,5.000000


In [83]:
rawdata.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5235 entries, 46 to 13258
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   City           5235 non-null   object
 1   Business Name  5235 non-null   object
 2   Review Text    5235 non-null   object
 3   Stars          5235 non-null   int64 
 4   TrueTimestamp  5235 non-null   object
dtypes: int64(1), object(4)
memory usage: 245.4+ KB


In [84]:
rawdata.head()

,City,Business Name,Review Text,Stars,TrueTimestamp
46,agadir,Attijariwafa Bank,Beaucoup des clients qui attends outside alors...,2,2022-03-14 21:22:17.360486
47,agadir,Attijariwafa Bank,Bon service,4,2023-03-14 21:22:17.360486
48,agadir,Attijariwafa Bank,Service👍🏽👍🏽👍🏽👍🏽👍🏽👍🏽 …,5,2024-12-13 21:22:17.360486
49,agadir,Attijariwafa Bank,No review text found,5,2012-03-16 21:22:17.360486
50,agadir,Attijariwafa Bank,"Le service est nul, j'ai pas de compte bancair...",1,2024-10-14 21:22:17.360486


SAVING TO A NEW CSV


In [85]:
rawdata.to_csv("data/updated_dataset.csv", index=False)

SPLITTING THE DATASET INTO 2 : ONE FOR STARS OTHER ONE FOR REVIEWS

In [86]:

# Split into two datasets
df_star = rawdata[["City", "Business Name", "Stars", "TrueTimestamp"]]  # Star Dataset
df_review = rawdata[["City", "Business Name", "Review Text", "TrueTimestamp"]]  # Review Dataset

# Save both datasets
df_star.to_csv("data/star_dataset.csv", index=False)
df_review.to_csv("data/review_dataset.csv", index=False)

# Show first few rows for confirmation
print("Star Dataset:")
print(df_star.head())

print("\nReview Dataset:")
print(df_review.head())

Star Dataset:
      City      Business Name  Stars               TrueTimestamp
46  agadir  Attijariwafa Bank      2  2022-03-14 21:22:17.360486
47  agadir  Attijariwafa Bank      4  2023-03-14 21:22:17.360486
48  agadir  Attijariwafa Bank      5  2024-12-13 21:22:17.360486
49  agadir  Attijariwafa Bank      5  2012-03-16 21:22:17.360486
50  agadir  Attijariwafa Bank      1  2024-10-14 21:22:17.360486

Review Dataset:
      City      Business Name  \
46  agadir  Attijariwafa Bank   
47  agadir  Attijariwafa Bank   
48  agadir  Attijariwafa Bank   
49  agadir  Attijariwafa Bank   
50  agadir  Attijariwafa Bank   

                                          Review Text  \
46  Beaucoup des clients qui attends outside alors...   
47                                        Bon service   
48                              Service👍🏽👍🏽👍🏽👍🏽👍🏽👍🏽 …   
49                               No review text found   
50  Le service est nul, j'ai pas de compte bancair...   

                 TrueTimestamp  
46  

1️⃣ Star Dataset → For numerical analysis of ratings.
2️⃣ Review Dataset → For NLP & sentiment analysis